<p style="font-weight:bold;"> <span style="font-size: 36px"> Extention Methods </span> </p

In [0]:
#!import "../DataModel/DataStructure"
#!import "ApplicationMessage"

# Enumerable Extensions

In [0]:
// because the default(T) is something else than the first/last element. What about "static T GetValidElement<T>(this IList<T> array, int index)"? 
static T GetElementOrDefault<T>(this ICollection<T> array, int index)
{   
    var count = array.Count;
    if (array == null || count == 0)
        return default(T);

    return index < 0
               ? array.ElementAt(0) // should this case be removed?
               : index < count
                   ? array.ElementAt(index)
                   : array.ElementAt(count -1);
}

In [0]:
public static Dictionary<TKey, TResult> ToDictionaryGrouped<TSource, TKey, TResult>(this IEnumerable<TSource> source, Func<TSource, TKey> keySelector, Func<IGrouping<TKey, TSource>, TResult> elementSelector) => source.GroupBy(keySelector).ToDictionary(g => g.Key, elementSelector);

In [0]:
public static IDataCube<TTarget> SelectToDataCube<TSource, TTarget>(this IEnumerable<TSource> source, Func<TSource, bool> whereClause, Func<TSource, TTarget> selector) => source.Where(whereClause).Select(selector).ToDataCube();

In [0]:
public static IDataCube<TTarget> SelectToDataCube<TSource, TTarget>(this IEnumerable<TSource> source, Func<TSource, TTarget> selector) => source.SelectToDataCube(x => true, selector);

In [0]:
public static double[] Prune (this IEnumerable<double> source, double precision = Precision) => source.Reverse().SkipWhile(x => Math.Abs(x) < precision).Reverse().ToArray();

# String Extensions

In [0]:
using System.Globalization;

In [0]:
public static double CheckStringForExponentialAndConvertToDouble (this string s)
{   
    if (s == null) return default;
    if (double.TryParse(s, NumberStyles.Number, CultureInfo.InvariantCulture, out var doubleValue)) return doubleValue;
    else { ApplicationMessage.Log(Error.ParsingScientificNotation, s); return 1; }
}

# Enum Extentions

In [0]:
public static bool Contains<T>(this T value, T lookingForFlag) 
    where T : struct
{
    int intValue = (int) (object) value;
    int intLookingForFlag = (int) (object) lookingForFlag;
    return ((intValue & intLookingForFlag) == intLookingForFlag);
}

# ToIdentityString

In [0]:
using System.Text;
public static string ToIdentityString<T>(this T v) where T : class
{
   StringBuilder sb = new StringBuilder();
   var propertyInfos = v.GetType()
                        .GetProperties()
                        .Where(x => Attribute.IsDefined(x, typeof(IdentityPropertyAttribute)))
                        .OrderByDescending(x => x.PropertyType.Name).ThenByDescending(x => x.Name)
                        .Select(x => sb.Append(x.Name).Append(":").Append(v.GetType().GetProperty(x.Name)?.GetValue(v, null)).Append(", ")).ToArray();
   return propertyInfos.Count() == 0? v.ToString() : propertyInfos.Select(p => p.ToString()).ToArray().Last();
}